# atmodeller

## Tutorial 1: basic operation

Import the required packages and set the package logger to the INFO level. For more output you could instead set it to DEBUG.

In [1]:
from atmodeller import InteriorAtmosphereSystem, Molecule, OCEAN_MOLES, MolarMasses, logger, FugacityConstraint, BufferedFugacityConstraint, SystemConstraint, MassConstraint
from atmodeller.thermodynamics import PeridotiteH2O, NoSolubility, BasaltDixonCO2, StandardGibbsFreeEnergyOfFormationHolland, StandardGibbsFreeEnergyOfFormationJANAF

import logging

logger.setLevel(logging.DEBUG)

16:19:51 - atmodeller                     - INFO      - atmodeller version 0.1.0


### 1. Simple H2O-H2 system with prescribed H2O pressure

We define a list of the molecules we wish to include in the interior-atmosphere system and how they partition between the melt and the atmosphere, and the solid and the melt:

In [2]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))

16:19:51 - atmodeller.thermodynamics      - INFO      - Creating a molecule: H2O
16:19:51 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'H': 2, 'O': 1}
16:19:51 - atmodeller.thermodynamics      - INFO      - Creating a molecule: H2
16:19:51 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'H': 2}


Although a choice is often made to constrain oxygen fugacity, we nevertheless need to explicitly include O2 as a molecule in the interior-atmosphere system:

In [3]:
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules

16:19:51 - atmodeller.thermodynamics      - INFO      - Creating a molecule: O2
16:19:51 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x1279b7090>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x130998110>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x128368250>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988)]

We can then create an interior-atmosphere system using the list of molecules and (optionally) specifying the thermodynamic data to use. Note that this creates a planet with 'default properties' (a molten Earth). Adjusting the planet properties will be covered in a later tutorial.

In [4]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules, gibbs_data=StandardGibbsFreeEnergyOfFormationJANAF())

16:19:51 - atmodeller.thermodynamics      - INFO      - Creating a new planet
16:19:51 - atmodeller.thermodynamics      - INFO      - Mantle mass (kg) = 4208261222595110885130240.000000
16:19:51 - atmodeller.thermodynamics      - INFO      - Mantle melt fraction = 1.000000
16:19:51 - atmodeller.thermodynamics      - INFO      - Core mass fraction = 0.295335
16:19:51 - atmodeller.thermodynamics      - INFO      - Planetary radius (m) = 6371000.000000
16:19:51 - atmodeller.thermodynamics      - INFO      - Planetary mass (kg) = 5972000000000000327155712.000000
16:19:51 - atmodeller.thermodynamics      - INFO      - Surface temperature (K) = 2000.000000
16:19:51 - atmodeller.thermodynamics      - INFO      - Surface gravity (m/s^2) = 9.819826
16:19:51 - atmodeller.thermodynamics      - INFO      - Melt Composition = None
16:19:51 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
16:19:51 - atmodeller.core                - INFO      - Molecules = ['H2

Notice that for this simple system it has identified the single reaction that relates the 3 chosen molecules. To solve the system, we provide a constraint of the H2O pressure in bar:

In [5]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
constraints: list = [H2O_pressure]

A second constraint is required to close the system of equations, and often this is constraining the fugacity of some species at a buffered value, such as constraining the oxygen fugacity at the iron-wustite (IW) buffer. Constraining the fO2 at the IW buffer is the default if no arguments are provided:

In [6]:
O2_pressure: SystemConstraint = BufferedFugacityConstraint()
constraints.append(O2_pressure)
logger.info(constraints)

16:19:51 - atmodeller                     - INFO      - [FugacityConstraint(species='H2O', value=1, field='fugacity'), BufferedFugacityConstraint(species='O2', fugacity=<atmodeller.thermodynamics.IronWustiteBufferOneill object at 0x13103b210>, log10_shift=0, field='fugacity')]


In [7]:
interior_atmosphere.solve(constraints)

16:19:51 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 BufferedFugacityConstraint(species='O2',
                            fugacity=<atmodeller.thermodynamics.IronWustiteBufferOneill object at 0x13103b210>,
                            log10_shift=0,
                            field='fugacity')]
16:19:51 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
16:19:51 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
16:19:51 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 H2 + 0.5 O2 = 1.0 H2O
16:19:52 - atmodeller.thermodynamics      - DEBUG     - Phase = <thermochem.janaf.JanafPhase object at 0x13113ca90>
  Hydrogen (H2)	H2(ref)
    Cp(298.15) = 28.836 J/mol/K
    S(298.15) = 130.680 J/mol/K
    [G-H(298.15)]/

{'H2': 1.4599206836641465, 'O2': 3.910491958127064e-08, 'H2O': 1.0}

You can access the solution directly using:

In [8]:
interior_atmosphere.fugacities_dict

{'H2': 1.4599206836641465, 'O2': 3.910491958127064e-08, 'H2O': 1.0}

You can access more information about the solution using:

In [9]:
interior_atmosphere.output

{'total_pressure_in_atmosphere': 2.4599207227690663,
 'mean_molar_mass_in_atmosphere': 0.008519831217100652,
 'H2': MoleculeOutput(mass_in_atmosphere=1.7941840016606449e+18, mass_in_solid=0.0, mass_in_melt=0.0, ppmw_in_solid=0.0, ppmw_in_melt=0.0, pressure_in_atmosphere=1.4599206836641465, volume_mixing_ratio=0.5934828184303246, mass_in_total=1.7941840016606449e+18),
 'O2': MoleculeOutput(mass_in_atmosphere=762878484272.4806, mass_in_solid=0.0, mass_in_melt=0.0, ppmw_in_solid=0.0, ppmw_in_melt=0.0, pressure_in_atmosphere=3.910491958127064e-08, volume_mixing_ratio=1.589682107204304e-08, mass_in_total=762878484272.4806),
 'H2O': MoleculeOutput(mass_in_atmosphere=1.0983211970337104e+19, mass_in_solid=0.0, mass_in_melt=2.205128880639838e+21, ppmw_in_solid=0.0, ppmw_in_melt=524.0, pressure_in_atmosphere=1.0, volume_mixing_ratio=0.4065171656728543, mass_in_total=2.2161120926101752e+21)}

### 2. System with C and H and prescribed pressures

We now extend the molecule list to additionally include C-species:

In [10]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a molecule: H2O
16:19:52 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'H': 2, 'O': 1}
16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a molecule: H2
16:19:52 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'H': 2}
16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a molecule: O2
16:19:52 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'O': 2}
16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a molecule: CO
16:19:52 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'C': 1, 'O': 1}
16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a molecule: CO2
16:19:52 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'C': 1, 'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x13111b3d0>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x111e49a50>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x131017b50>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x128052150>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, molar_mass=0.0280101),
 Molecule(name='CO2', solubility=<atmodeller.thermodynamics.BasaltDixonCO2 object at 0x13115fd10>, 

In [11]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)

16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a new planet
16:19:52 - atmodeller.thermodynamics      - INFO      - Mantle mass (kg) = 4208261222595110885130240.000000
16:19:52 - atmodeller.thermodynamics      - INFO      - Mantle melt fraction = 1.000000
16:19:52 - atmodeller.thermodynamics      - INFO      - Core mass fraction = 0.295335
16:19:52 - atmodeller.thermodynamics      - INFO      - Planetary radius (m) = 6371000.000000
16:19:52 - atmodeller.thermodynamics      - INFO      - Planetary mass (kg) = 5972000000000000327155712.000000
16:19:52 - atmodeller.thermodynamics      - INFO      - Surface temperature (K) = 2000.000000
16:19:52 - atmodeller.thermodynamics      - INFO      - Surface gravity (m/s^2) = 9.819826
16:19:52 - atmodeller.thermodynamics      - INFO      - Melt Composition = None
16:19:52 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
16:19:52 - atmodeller.core                - INFO      - Molecules = ['CO

Note now the system has identified two reactions in the network. With C present in the system we must provide at least 2 constraints, in addition to the oxygen fugacity:

In [12]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
CO2_pressure: SystemConstraint = FugacityConstraint(species='CO2', value=1)
O2_pressure: SystemConstraint = BufferedFugacityConstraint()
constraints: list[SystemConstraint] = [H2O_pressure, CO2_pressure, O2_pressure]

In [13]:
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

16:19:52 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 FugacityConstraint(species='CO2', value=1, field='fugacity'),
 BufferedFugacityConstraint(species='O2',
                            fugacity=<atmodeller.thermodynamics.IronWustiteBufferOneill object at 0x13115fc50>,
                            log10_shift=0,
                            field='fugacity')]
16:19:52 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
16:19:52 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
16:19:52 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
16:19:52 - atmodeller.thermodynamics      - DEBUG     - Phase = <thermochem.janaf.JanafPhase object at 0x13116be10>
  Carbon Monoxide (CO)	C1O1(g)
    Cp(298.1

{'CO': 6.656248374677096,
 'H2': 1.4599206836641465,
 'O2': 3.910491958127064e-08,
 'CO2': 1.0,
 'H2O': 1.0}

There is not a requirement to necessarily impose the oxygen fugacity as a constraint. Instead, we can simply impose three pressure constraints (that span the reaction set) and allow for the oxygen fugacity to be solved. Note that if we do not specify an appropriate range of constraints we cannot solve the system of equations to give a unique solution and hence the code will raise an exception relating to a singular matrix.

In [14]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
H2_pressure: SystemConstraint = FugacityConstraint(species='H2', value=1)
CO_pressure: SystemConstraint = FugacityConstraint(species='CO', value=1)
constraints: list[SystemConstraint] = [H2O_pressure, H2_pressure, CO_pressure]

In [15]:
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

16:19:52 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 FugacityConstraint(species='H2', value=1, field='fugacity'),
 FugacityConstraint(species='CO', value=1, field='fugacity')]
16:19:52 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
16:19:52 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
16:19:52 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
16:19:52 - atmodeller.thermodynamics      - DEBUG     - Phase = <thermochem.janaf.JanafPhase object at 0x131195250>
  Carbon Monoxide (CO)	C1O1(g)
    Cp(298.15) = 29.142 J/mol/K
    S(298.15) = 197.653 J/mol/K
    [G-H(298.15)]/298.15 = 197.653 J/mol/K
    H-H(298.15) = 0.000 J/mol/K
    Delta_fH(298.15) = -110527 J/mol
    Delta_fG(298.15

{'CO': 1.0,
 'H2': 1.0,
 'O2': 8.33469899813586e-08,
 'CO2': 0.21933086049166114,
 'H2O': 1.0}

### 3. System with C and H and mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of pressure and mass constraints. We define the same molecule set as before:

In [16]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a molecule: H2O
16:19:52 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'H': 2, 'O': 1}
16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a molecule: H2
16:19:52 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'H': 2}
16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a molecule: O2
16:19:52 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'O': 2}
16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a molecule: CO
16:19:52 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'C': 1, 'O': 1}
16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a molecule: CO2
16:19:52 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'C': 1, 'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x131163510>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x1311633d0>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x131163110>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x131162c10>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, molar_mass=0.0280101),
 Molecule(name='CO2', solubility=<atmodeller.thermodynamics.BasaltDixonCO2 object at 0x131038e90>, 

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [17]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = number_of_earth_oceans * OCEAN_MOLES * MolarMasses().H2
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    BufferedFugacityConstraint()
]

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

16:19:52 - atmodeller.thermodynamics      - INFO      - Creating a new planet
16:19:52 - atmodeller.thermodynamics      - INFO      - Mantle mass (kg) = 4208261222595110885130240.000000
16:19:52 - atmodeller.thermodynamics      - INFO      - Mantle melt fraction = 1.000000
16:19:52 - atmodeller.thermodynamics      - INFO      - Core mass fraction = 0.295335
16:19:52 - atmodeller.thermodynamics      - INFO      - Planetary radius (m) = 6371000.000000
16:19:52 - atmodeller.thermodynamics      - INFO      - Planetary mass (kg) = 5972000000000000327155712.000000
16:19:52 - atmodeller.thermodynamics      - INFO      - Surface temperature (K) = 2000.000000
16:19:52 - atmodeller.thermodynamics      - INFO      - Surface gravity (m/s^2) = 9.819826
16:19:52 - atmodeller.thermodynamics      - INFO      - Melt Composition = None
16:19:52 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
16:19:52 - atmodeller.core                - INFO      - Molecules = ['CO

{'CO': 62.354011354608105,
 'H2': 0.5735474798340308,
 'O2': 3.910491958127064e-08,
 'CO2': 9.367741082473197,
 'H2O': 0.39286208233897085}

### 4. Including more species such as CH4

It is straightforward to add more species to the system, although they must have their formation energies and masses already specified in the code.

In [18]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CH4', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules

16:19:53 - atmodeller.thermodynamics      - INFO      - Creating a molecule: H2O
16:19:53 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'H': 2, 'O': 1}
16:19:53 - atmodeller.thermodynamics      - INFO      - Creating a molecule: H2
16:19:53 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'H': 2}
16:19:53 - atmodeller.thermodynamics      - INFO      - Creating a molecule: O2
16:19:53 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'O': 2}
16:19:53 - atmodeller.thermodynamics      - INFO      - Creating a molecule: CO
16:19:53 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'C': 1, 'O': 1}
16:19:53 - atmodeller.thermodynamics      - INFO      - Creating a molecule: CO2
16:19:53 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'C': 1, 'O': 2}
16:19:53 - atmodeller.thermodynamics      - INFO      - Creating a molecule: CH4
16:19:53 - atmodeller.thermodynamics      - DEBUG     - element count = 
{'C': 1, '

[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x131184f50>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x1311378d0>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x131036f90>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x131035a10>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, molar_mass=0.0280101),
 Molecule(name='CO2', solubility=<atmodeller.thermodynamics.BasaltDixonCO2 object at 0x131036610>, 

We define a mixture of mass and oxygen fugacity constraints as before and solve the system. CH4 is not prevalent at 2000 K so the results are almost identical to those without CH4 presented above.

In [19]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = number_of_earth_oceans * OCEAN_MOLES * MolarMasses().H2
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    BufferedFugacityConstraint()
]

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

16:19:53 - atmodeller.thermodynamics      - INFO      - Creating a new planet
16:19:53 - atmodeller.thermodynamics      - INFO      - Mantle mass (kg) = 4208261222595110885130240.000000
16:19:53 - atmodeller.thermodynamics      - INFO      - Mantle melt fraction = 1.000000
16:19:53 - atmodeller.thermodynamics      - INFO      - Core mass fraction = 0.295335
16:19:53 - atmodeller.thermodynamics      - INFO      - Planetary radius (m) = 6371000.000000
16:19:53 - atmodeller.thermodynamics      - INFO      - Planetary mass (kg) = 5972000000000000327155712.000000
16:19:53 - atmodeller.thermodynamics      - INFO      - Surface temperature (K) = 2000.000000
16:19:53 - atmodeller.thermodynamics      - INFO      - Surface gravity (m/s^2) = 9.819826
16:19:53 - atmodeller.thermodynamics      - INFO      - Melt Composition = None
16:19:53 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
16:19:53 - atmodeller.core                - INFO      - Molecules = ['CO

{'CO': 62.35400971598908,
 'H2': 0.5735474727549934,
 'O2': 3.910491958127064e-08,
 'CO2': 9.36774083629563,
 'H2O': 0.39286207749005125,
 'CH4': 1.3473271903639385e-06}